In [31]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

In [32]:
start = dt.datetime(2000,1,1)
end = dt.datetime.now()
# start = dt.datetime(2019,3,1)
# end = dt.datetime(2022,3,1)
start,end

(datetime.datetime(2000, 1, 1, 0, 0),
 datetime.datetime(2022, 6, 15, 6, 19, 34, 108483))

In [33]:
stock = 'NFLX'

In [34]:
df = pdr.get_data_yahoo(stock,start,end)
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2002-05-23,1.242857,1.145714,1.156429,1.196429,104790000.0,1.196429
2002-05-24,1.225000,1.197143,1.214286,1.210000,11104800.0,1.210000
2002-05-28,1.232143,1.157143,1.213571,1.157143,6609400.0,1.157143
2002-05-29,1.164286,1.085714,1.164286,1.103571,6757800.0,1.103571
2002-05-30,1.107857,1.071429,1.107857,1.071429,10154200.0,1.071429
...,...,...,...,...,...,...
2022-06-08,207.380005,197.639999,197.949997,202.830002,7244100.0,202.830002
2022-06-09,201.690002,192.539993,200.649994,192.770004,6949000.0,192.770004
2022-06-10,185.860001,180.850006,182.639999,182.940002,13065600.0,182.940002


In [35]:
target_y = df.Close
target_y

Date
2002-05-23      1.196429
2002-05-24      1.210000
2002-05-28      1.157143
2002-05-29      1.103571
2002-05-30      1.071429
                 ...    
2022-06-08    202.830002
2022-06-09    192.770004
2022-06-10    182.940002
2022-06-13    169.690002
2022-06-14    167.539993
Name: Close, Length: 5051, dtype: float64

In [36]:
target_y.plot()

In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM                        
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [38]:
X_feat=df.iloc[:,0:3]
target_y = df.Close

In [39]:
#Feature Scaling
sc = StandardScaler()
X_ft=sc.fit_transform(X_feat.values)
X_ft=pd.DataFrame(columns=X_feat.columns,
                    data=X_ft,
                    index=X_feat.index)

In [40]:
def lstm_split(data,n_steps):
    X, y=[],[]
    for i in range(len(data)-n_steps+1):
        X.append(data[i:i+n_steps,:-1])
        y.append(data[i+n_steps-1,-1])
    return np.array(X),np.array(y)


In [41]:
X1, y1 = lstm_split(df.values,n_steps=2)
train_split=0.8
split_idx=int(np.ceil(len(X1)*train_split))
date_index=df.index
X_train,X_test=X1[:split_idx],X1[split_idx:]
y_train,y_test=y1[:split_idx],y1[split_idx:]
X_train_date,X_test_date=date_index[:split_idx],date_index[split_idx:]
print(X1.shape,X_train.shape,X_test.shape,y_test.shape)

(5050, 2, 5) (4040, 2, 5) (1010, 2, 5) (1010,)


In [42]:
lstm=Sequential()
lstm.add(LSTM(32,input_shape=(X_train.shape[1],X_train.shape[2]),
              activation='relu',return_sequences=True))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error',optimizer='adam')
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 2, 32)             4864      
                                                                 
 dense_1 (Dense)             (None, 2, 1)              33        
                                                                 
Total params: 4,897
Trainable params: 4,897
Non-trainable params: 0
_________________________________________________________________


In [43]:
history=lstm.fit(X_train,y_train,
                 epochs=100,batch_size=4,
                 verbose=2,shuffle=False)

Epoch 1/100
1010/1010 - 1s - loss: 71777435648.0000 - 1s/epoch - 1ms/step
Epoch 2/100
1010/1010 - 1s - loss: 49226530816.0000 - 994ms/epoch - 984us/step
Epoch 3/100
1010/1010 - 1s - loss: 31756820480.0000 - 1s/epoch - 1ms/step
Epoch 4/100
1010/1010 - 1s - loss: 24676067328.0000 - 1s/epoch - 1ms/step
Epoch 5/100
1010/1010 - 1s - loss: 5327475200.0000 - 1s/epoch - 1ms/step
Epoch 6/100
1010/1010 - 1s - loss: 5045223936.0000 - 1s/epoch - 999us/step
Epoch 7/100
1010/1010 - 1s - loss: 10339127296.0000 - 1s/epoch - 1ms/step
Epoch 8/100
1010/1010 - 1s - loss: 6467287552.0000 - 1s/epoch - 1ms/step
Epoch 9/100
1010/1010 - 1s - loss: 4858583040.0000 - 1s/epoch - 1ms/step
Epoch 10/100
1010/1010 - 1s - loss: 3709489408.0000 - 1s/epoch - 1ms/step
Epoch 11/100
1010/1010 - 1s - loss: 3428734208.0000 - 1s/epoch - 993us/step
Epoch 12/100
1010/1010 - 1s - loss: 2746539264.0000 - 1000ms/epoch - 990us/step
Epoch 13/100
1010/1010 - 1s - loss: 2125537024.0000 - 1s/epoch - 1ms/step
Epoch 14/100
1010/1010 - 1s

In [53]:
y_pred=lstm.predict(X_test)

In [54]:
rmse = mean_squared_error(y_test,y_pred,squared=False)
mape = mean_absolute_percentage_error(y_test,y_pred)
print("RSME:",rmse)
print("MAPE:",mape)

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
lstm=Sequential()
lstm.add(LSTM(50,input_shape=(X_train.shape[1],X_train.shape[2]),
        activation='relu',return_sequences=True))
lstm.add(LSTM(50,activation='relu'))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error',optimizer='adam')
lstm.summary()

In [ ]:
rmse = mean_squared_error(y_test,y_pred,squared=False)
mape = mean_absolute_percentage_error(y_test,y_pred)
print("RSME:",rmse)
print("MAPE:",mape)

In [ ]:
n_steps=10
X1,y1=lstm_split(df.values,n_steps=n_steps)
train_split=0.8
split_idx=int(np.ceil(len(X1)*train_split))
date_index=df.index
X_train,x_test=X1[:split_idx],X1[split_idx:]
y_train,y_test=y1[:split_idx],y1[split_idx:]
X_train_date,X_test_date=date_index[:split_idx],date_index[split_idx:-n_steps]
print(X1.shape,X_train.shape,x_test.shape,X_test_date.shape,y_test.shape)

In [ ]:
rmse=mean_squared_error(y_test,y_pred,squared=False)
mape=mean_absolute_percentage_error(y_test,y_pred)

print("RSME:",rmse)
print("MAPE:",mape)